# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# !pip install cartopy geoviews pyproj

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.88,77,100,4.87,PN,1699150162
1,1,luderitz,-26.6481,15.1594,14.24,85,18,4.92,NaN,1699150162
2,2,college,64.8569,-147.8028,-5.03,65,100,5.14,US,1699150162
3,3,uturoa,-16.7333,-151.4333,25.41,84,90,7.99,PF,1699150163
4,4,kuyucak,37.9133,28.4592,17.35,93,58,1.52,TR,1699150163


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    color = "City",
    alpha = 0.60,
    size = "Humidity",
    tiles = "OSM"
)

# Display the map
map1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
#city_data_df["Max Temp"] = city_data_df["Max Temp"] - 273.15

city_data_df = city_data_df[city_data_df["Max Temp"] >= 15]
city_data_df = city_data_df[city_data_df["Max Temp"] <= 30]

# max_temp = 30
# min_temp = 15

# # Drop any rows with null values
city_data_df.dropna(inplace=True)

# # Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.88,77,100,4.87,PN,1699150162
3,3,uturoa,-16.7333,-151.4333,25.41,84,90,7.99,PF,1699150163
4,4,kuyucak,37.9133,28.4592,17.35,93,58,1.52,TR,1699150163
5,5,freetown,8.4840,-13.2299,26.43,84,100,2.13,SL,1699150163
7,7,blackmans bay,-43.0167,147.3167,15.07,62,14,4.01,AU,1699150164
...,...,...,...,...,...,...,...,...,...,...
573,573,puerto colombia,10.9878,-74.9547,27.05,75,3,3.30,CO,1699150380
574,574,awjilah,29.1081,21.2869,23.68,29,17,3.49,LY,1699150380
576,576,shenjiamen,29.9576,122.2980,23.76,81,10,9.73,CN,1699150381
577,577,dondo,-19.6094,34.7431,17.76,85,100,1.90,MZ,1699150381


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,adamstown,-25.0660,-130.1015,22.88,77,100,4.87,PN,1699150162,
1,1,luderitz,-26.6481,15.1594,14.24,85,18,4.92,NaN,1699150162,
2,2,college,64.8569,-147.8028,-5.03,65,100,5.14,US,1699150162,
3,3,uturoa,-16.7333,-151.4333,25.41,84,90,7.99,PF,1699150163,
4,4,kuyucak,37.9133,28.4592,17.35,93,58,1.52,TR,1699150163,
...,...,...,...,...,...,...,...,...,...,...,...
574,574,awjilah,29.1081,21.2869,23.68,29,17,3.49,LY,1699150380,
575,575,montana,43.4125,23.2250,11.12,72,0,3.32,BG,1699150380,
576,576,shenjiamen,29.9576,122.2980,23.76,81,10,9.73,CN,1699150381,
577,577,dondo,-19.6094,34.7431,17.76,85,100,1.90,MZ,1699150381,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories" : "accommodation.hotel",
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{row['Lng']},{radius} "
    params["bias"] = f"proximity:{row['Lng']},{row['Lat']}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError) as e:
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
luderitz - nearest hotel: No hotel found
college - nearest hotel: No hotel found
uturoa - nearest hotel: No hotel found
kuyucak - nearest hotel: No hotel found
freetown - nearest hotel: No hotel found
nar'yan-mar - nearest hotel: No hotel found
blackmans bay - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
shimoda - nearest hotel: No hotel found
longyearbyen - nearest hotel: No hotel found
aleg - nearest hotel: No hotel found
bubaque - nearest hotel: No hotel found
mudhol - nearest hotel: No hotel found
hadibu - nearest hotel: No hotel found
invercargill - nearest hotel: No hotel found
road town - nearest hotel: No hotel found
mar del plata - nearest hotel: No hotel found
badger - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
ust'-ilimsk - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
jiuquan - nearest hotel: No hotel found
edinburgh o

odemira - nearest hotel: No hotel found
swift current - nearest hotel: No hotel found
khanty-mansiysk - nearest hotel: No hotel found
kuytun - nearest hotel: No hotel found
aras-asan - nearest hotel: No hotel found
tazovsky - nearest hotel: No hotel found
severo-yeniseyskiy - nearest hotel: No hotel found
coahuayana de hidalgo - nearest hotel: No hotel found
hami - nearest hotel: No hotel found
grand falls-windsor - nearest hotel: No hotel found
constantia - nearest hotel: No hotel found
nijar - nearest hotel: No hotel found
mugia - nearest hotel: No hotel found
ouahigouya - nearest hotel: No hotel found
korla - nearest hotel: No hotel found
idil - nearest hotel: No hotel found
ust-nera - nearest hotel: No hotel found
okhotsk - nearest hotel: No hotel found
port lincoln - nearest hotel: No hotel found
willmar - nearest hotel: No hotel found
magadan - nearest hotel: No hotel found
tobol'sk - nearest hotel: No hotel found
papagaios - nearest hotel: No hotel found
kerikeri - nearest hotel

la'ie - nearest hotel: No hotel found
mombetsu - nearest hotel: No hotel found
csorna - nearest hotel: No hotel found
sanming - nearest hotel: No hotel found
pedernales - nearest hotel: No hotel found
camiri - nearest hotel: No hotel found
tawau - nearest hotel: No hotel found
codrington - nearest hotel: No hotel found
gamba - nearest hotel: No hotel found
hampton bays - nearest hotel: No hotel found
itogon - nearest hotel: No hotel found
westport - nearest hotel: No hotel found
vilyuchinsk - nearest hotel: No hotel found
saran - nearest hotel: No hotel found
new norfolk - nearest hotel: No hotel found
devonport - nearest hotel: No hotel found
georgievka - nearest hotel: No hotel found
valparaiso - nearest hotel: No hotel found
mandiana - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
murzuq - nearest hotel: No hotel found
waingapu - nearest hotel: No hotel found
nantucket - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
ciudad lazar

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,0,adamstown,-25.0660,-130.1015,22.88,77,100,4.87,PN,1699150162,No hotel found
1,1,luderitz,-26.6481,15.1594,14.24,85,18,4.92,NaN,1699150162,No hotel found
2,2,college,64.8569,-147.8028,-5.03,65,100,5.14,US,1699150162,No hotel found
3,3,uturoa,-16.7333,-151.4333,25.41,84,90,7.99,PF,1699150163,No hotel found
4,4,kuyucak,37.9133,28.4592,17.35,93,58,1.52,TR,1699150163,No hotel found
...,...,...,...,...,...,...,...,...,...,...,...
574,574,awjilah,29.1081,21.2869,23.68,29,17,3.49,LY,1699150380,No hotel found
575,575,montana,43.4125,23.2250,11.12,72,0,3.32,BG,1699150380,No hotel found
576,576,shenjiamen,29.9576,122.2980,23.76,81,10,9.73,CN,1699150381,No hotel found
577,577,dondo,-19.6094,34.7431,17.76,85,100,1.90,MZ,1699150381,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.75,
    hover_cols = ["Hotel Name" , "Country"]

)

# Display the map
map2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)